In [2]:
from pandas_datareader import data
import pandas_datareader as pdr
import pandas as pd
import numpy as np
import talib as ta1
import pandas_ta as ta
import datetime
import requests

import warnings
warnings.filterwarnings('ignore')
warnings.warn('DelftStack')
warnings.warn('Do not show this message')


# Retrieve Trading data from houbi , 1 min interval

In [3]:
ticker = 'ethusdt'
base = 'https://api.huobi.pro'
url = '/market/history/kline?period=60min&size=2000&symbol='+ ticker
final_url = base + url

reqs = requests.get(final_url)

df = pd.DataFrame(reqs.json()['data'])
df["Date"] = pd.to_datetime(df["id"], unit='s', origin='unix')
df["Date"] = df["Date"].dt.strftime("%m/%d/%Y %H:%M:%S")
df.drop('id',axis = 1, inplace = True)
df['Date'] = pd.to_datetime(df['Date'])
df.set_index('Date',inplace = True)
df.rename(columns={'high':'High','low':'Low','open':'Open','close':'Close','vol':'Volume'},inplace=True)
df['Adj Close']= df['Close']
#df = df[['High','Low','Open','Close','Volume','Adj Close']]
df.sort_index(inplace = True)



# RSI indicator

In [4]:
df['RSI'] = ta.rsi(df['Close'],Length = 7)
df['RSI<30'] = df['RSI'].apply(lambda x:  1 if x <= 30 else 0 )
df['RSI>70'] = df['RSI'].apply(lambda x:  1 if x >= 70 else 0 )

# Create DAC
- Buy lower when price goes lower then initial buy price
    - 1st   : x price < then RSI 30
    - subsequence 1% lower : buy 50 percent of previous price
    - sell while price above 1% of average buy price



In [25]:
buying = False
N = 8 # max buying
alocate_cost = (N * 4)+1 #break to 33 times
df['Budget'] = 100
df['Cost'] = np.nan
df['Average Cost'] = np.nan

for x in range(len(df)):
    if df['RSI<30'][x] == 1 and buying == False:
        buy_datetime = df.index[x]
        df['Cost'][x]  = df['Budget'][x-1] # lock down the price when enter
        #subsequence price & alocation cost 
        buy_price_1 , budget_1= df['Close'][x]*0.99 , df['Cost'][x] * 1/alocate_cost
        buy_price_2 , budget_2= buy_price_1*0.99 , df['Cost'][x] * 1/alocate_cost
        buy_price_3 , budget_3= buy_price_2*0.99 , df['Cost'][x] * 2/alocate_cost
        buy_price_4 , budget_4= buy_price_3*0.99 , df['Cost'][x] * 2/alocate_cost
        buy_price_5 , budget_5= buy_price_4*0.99 , df['Cost'][x] * 3/alocate_cost
        buy_price_6 , budget_6= buy_price_5*0.99 , df['Cost'][x] * 5/alocate_cost
        buy_price_7 , budget_7= buy_price_6*0.99 , df['Cost'][x] * 8/alocate_cost
        buy_price_8 , budget_8= buy_price_7*0.99 , df['Cost'][x] * 11/alocate_cost
        buying = True
        #print(buy_datetime)
        #print(buy_price)
        #print(buy_price_2,budget_2)
        #print(buy_price_3,budget_3)
        #print(buy_price_4,budget_4)
        #print(buy_price_5,budget_5)
        #print(buy_price_6,budget_6)
        #print(buy_price_7,budget_7)
        #print(buy_price_8,budget_8)     
        #print(budget_1+budget_2+budget_3+budget_4+budget_5+budget_6+budget_7+budget_8)

    if buying == True : ##DAC
        ## calculate average price 
        if df['Low'][x] <= buy_price_8:
            N = N -1 

        elif df['Low'][x] <= buy_price_7:
            print[df['Low'][x]]

        elif df['Low'][x] <= buy_price_6:
            print[df['Low'][x]]

        elif df['Low'][x] <= buy_price_5:
            print[df['Low'][x]]

        elif df['Low'][x] <= buy_price_4:
            print[df['Low'][x]]

        elif df['Low'][x] <= buy_price_3:
            print[df['Low'][x]]

        elif df['Low'][x] <= buy_price_2:
            print[df['Low'][x]]

        elif df['Low'][x] <= buy_price_1:
            print[df['Low'][x]]




        buy_price = df['Close'][x]
        #TP_price = buy_price * 1.01 # buy_price need to update 
        
        buying = True

    if buying == True and df['Close'][x] > TP_price: 
        df['Budget'][x] = 1
        buying = False


2679.936435 3.0303030303030303
2653.13707065 6.0606060606060606
2626.6056999435 6.0606060606060606
2600.339642944065 9.090909090909092
2574.3362465146242 15.151515151515152
2548.592884049478 24.242424242424242
2523.1069552089834 33.333333333333336
100.0


NameError: name 'TP_price' is not defined

# Trading Condition
- Long signal when RSI value below 30 
- Short signal when RSI value above 70

# Analysis

In [19]:
# Number of RSI below 30 signal 
print(max(df.index) - min(df.index))
df['RSI<30'].value_counts()

83 days 07:00:00


0    1873
1     127
Name: RSI<30, dtype: int64